<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2
from utils import BBANDS_signal, Signal
from talib import BBANDS
from numba import njit

gazp = pd.read_csv("gazp.csv", parse_dates=["Date"]).sort_values("Date")
gazp = gazp[:150]
date = gazp['Date']
price = gazp["Close"].values
upper, middle, lower = BBANDS(price, 20, 2, 2, matype=0)

In [ ]:
Strategy(
    df = df,
    enter_long=[('Close', 'lower', 'crossup', False),],
    exit_long=[
        ('Close', 'middle', 'crossup', False),
        'or',
        ('Close', 'lower', 'crossdown', False),
    ],
    enter_short=('Close', 'upper', 'crossdown', False),
    exit_short=[
        ('Close', 'middle', 'crossdown', False),
        'or',
        ('Close', 'upper', 'crossup', False),
    ],
)

In [3]:
gazp['upper'], gazp['middle'], gazp['lower'] = BBANDS(price, 20,2,2)

In [53]:
def check_tuple(obj, df):
    if len(obj) != 4:
        raise ValueError(f'signal {obj} must consist of 4 elements: 2 cols, a signal type, and a bool')
    if obj[0] not in df.columns or obj[1] not in df.columns:
        raise ValueError(f'first 2 elements of signal {obj} must be columns in input df')
    if obj[2] not in ['crossup', 'crossdown','higher','lower']:
        raise ValueError(f"signal type in {obj} must be one of 'crossup', 'crossdown','higher', or 'lower'")
    if not(isinstance(obj[3], bool)):
        raise ValueError(f'signal accumulator field in {obj} must be boolean')
        
def check_list(obj, df):
    if (len(obj) - 1)%2 != 0:
        raise ValueError(f'signal {obj} must have total length of 3, or 5, or 7, etc elements')
    for i,el in enumerate(obj):
        if i%2 == 0:
            check_tuple(el, df)
        else:
            if el not in ['and','or']:
                raise ValueError(f'elemnts of signal {obj} must be divided by "and" or "or"')

def check_signal(obj, df):
    
    if not (isinstance(obj, tuple) or isinstance(obj, list)):
        raise ValueError(f'signal {obj} must be either tuple or list')
    
    elif isinstance(obj, tuple):
        check_tuple(obj, df)
        obj = [obj]
    
    elif isinstance(obj, list) and len(obj) == 1:
        check_tuple(obj[0], df)
    
    else:
        check_list(obj, df)   
        
    return obj

In [ ]:
def parse_tuple(tpl, df):
    return Signal(df[tpl[0]].values, df[tpl[1]].values, tpl[2], tpl[3])

def parse_signal(obj, df):
    f

In [43]:
sig1 = Signal(gazp['Close'].values, gazp['middle'].values,'crossup', False)
sig2 = Signal(gazp['Close'].values, gazp['lower'].values, 'crossdown', False)

In [52]:
X = {'a':True, 'b':False}
x = ("a","and","b")
y = ("a","or","b")

def func(z):
    return X[z[0]] and/or X[z[2]]

'and'

In [4]:
class Strategy:
    
    def __init__(self, df, enter_long, exit_long, enter_short, exit_short, commission=.0003):
        self.df = df
        self.enter_long = check_signal(enter_long)
        self.exit_long = check_signal(exit_long)
        self.enter_short = check_signal(enter_short)
        self.exit_short = check_signal(exit_short)
        self.commission = commission
        
    
    @njit
    def run(self):
        enter_long = Signal(self.enter_long)
        exit_long = Signal(self.exit_long)
        enter_short = Signal(self.enter_short)
        exit_short = Signal(self.exit_short)

        L = self.price.shape[0]
        out = np.empty(L)
        out[:2] = 0.0
        out[-2:] = 0.0
        i = 1

        while i < (L-2):
            

            out[i+1] = 0.0

            if enter_long.sig(i) and i < (L-2):
                out[i+1] = self.price[i+2]/(self.price[i+1]*(1+self.commission)) - 1.0
                trade = True
                while trade and i < (L-3):
                    i += 1
                    out[i+1] = self.price[i+2]/self.price[i+1] - 1.0
                    if exit_long.sig(i):
                        trade = False
                        out[i+1] = (1-self.commission)*self.price[i+2]/self.price[i+1] - 1.0


            if enter_short(i) and i < (L-2):
                out[i+1] = (1-self.commission)*self.price[i+1]/self.price[i+2] - 1.0
                trade = True
                while trade and i < (L-3):
                    i += 1
                    out[i+1] = self.price[i+1]/self.price[i+2] - 1.0
                    if exit_short(i):
                        trade = False
                        out[i+1] = self.price[i+1]/(self.price[i+2]*(1+self.commission)) - 1.0


            i +=1
        return out